<a href="https://colab.research.google.com/github/dnguyend/rayleigh_newton/blob/master/colab/ZPairsEigenTensor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

$\newcommand{\cT}{\mathcal{T}}$
$\newcommand{\cB}{\mathcal{B}}$
$\newcommand{\C}{\mathbb{C}}$
$\newcommand{\R}{\mathbb{R}}$
# This workbook compute all complex eigentensor pairs by Rayleigh quotient iteration
The problem is
$$\cT(I, X^{[m-1]}) -X\lambda = 0 $$
The square bracket means the number of times $X$ is repeated.
* $\cT$ and is a real $(m, n)$ tensors
* $X\in \C^n, \lambda\in \R$. 
* The number of eigenvalues is $\sum_{i=0}^{n-1}(m-1)^i$, or $\frac{(m-1)^n-1}{m-2}$ if $m\neq 2$.
* An extension of the Rayleigh quotient iteration. We also implement the Rayleigh-Chebyshev algorithm.

First, clone the project from github


In [1]:
!git clone https://github.com/dnguyend/rayleigh_newton

Cloning into 'rayleigh_newton'...
remote: Enumerating objects: 249, done.
remote: Counting objects: 100% (249/249), done.
remote: Compressing objects: 100% (118/118), done.
remote: Total 249 (delta 130), reused 244 (delta 125), pack-reused 0
Receiving objects: 100% (249/249), 14.73 MiB | 9.14 MiB/s, done.
Resolving deltas: 100% (130/130), done.


Importing main functions to be used later - but the code to find all complex eigen pairs is in the next block.

In [2]:
from __future__ import print_function
import numpy as np
import pandas as pd
from time import process_time
import rayleigh_newton.core.utils as utils

from rayleigh_newton.core.eigen_tensor_solver import\
    orthogonal_newton_correction_method, schur_form_rayleigh,\
    symmetric_tv_mode_product, schur_form_rayleigh_chebyshev,\
    newton_form_rayleigh_chebyshev


In [3]:
import numpy as np
import sys
import scipy.linalg
from numpy import concatenate, tensordot, eye, zeros, zeros_like,\
    power, sqrt, exp, pi

from numpy.linalg import solve, inv, norm

if sys.version_info[0] < 3:
    class SimpleNamespace:
        def __init__(self, **kwargs):
            self.__dict__.update(kwargs)

            def __repr__(self):
                keys = sorted(self.__dict__)
                items = ("{}={!r}".format(k, self.__dict__[k]) for k in keys)
                return "{}({})".format(type(self).__name__, ", ".join(items))

            def __eq__(self, other):
                return self.__dict__ == other.__dict__
else:
    from types import SimpleNamespace


def symmetric_tv_mode_product(T, x, modes) -> np.ndarray:
    v = T
    for i in range(modes):
        v = tensordot(v, x, axes=1)
    return v

def schur_form_rayleigh_chebyshev_unitary(
        T, max_itr, delta, x_init=None, do_chebyshev=True):
    """Schur form rayleigh chebyshev unitary
    T and x are complex. Constraint is x^H x = 1
    lbd is real
    """
    # get tensor dimensionality and order
    n_vec = T.shape
    m = len(n_vec)
    n = T.shape[0]
    R = 1
    converge = False

    # if not given as input, randomly initialize
    if x_init is None:
        x_init = np.random.randn(n)
        x_init = x_init/norm(x_init)

    # init lambda_(k) and x_(k)
    x_k = x_init.copy()
    if do_chebyshev:
        T_x_m_3 = symmetric_tv_mode_product(T, x_k, m-3)
        T_x_m_2 = tensordot(T_x_m_3, x_k, axes=1)

    else:
        T_x_m_2 = symmetric_tv_mode_product(T, x_k, m-2)
    T_x_m_1 = T_x_m_2 @ x_k

    lbd = (x_k.conjugate().T @ T_x_m_1).real
    ctr = 0

    while (R > delta) and (ctr < max_itr):
        # compute T(I,I,x_k,...,x_k), T(I,x_k,...,x_k) and g(x_k)
        rhs = concatenate(
            [x_k.reshape(-1, 1), T_x_m_1.reshape(-1, 1)], axis=1)

        # compute Hessian H(x_k)
        H = (m-1)*T_x_m_2-lbd*eye(n)
        lhs = solve(H, rhs)

        # fix eigenvector
        y = lhs[:, 0] * (
            np.sum((x_k.conjugate() * lhs[:, 1]).real) /
            np.sum((x_k.conjugate() * lhs[:, 0]).real)) - lhs[:, 1]
        if do_chebyshev and (np.linalg.norm(y) < 30e-2):
            J_R_eta = y.conjugate().T @ T_x_m_1 +\
                      (m-1) * x_k.conjugate().T @ T_x_m_2 @ y -\
                      2*(x_k.conjugate().T @ y)*lbd
            L_x_lbd = -y * J_R_eta
            L_x_x = (m-1) * (m-2) * np.tensordot(T_x_m_3, y, axes=1) @ y
            T_a = np.linalg.solve(H, -L_x_lbd - 0.5 * L_x_x)
            T_adj = T_a - lhs[:, 0] *\
                np.sum((x_k.conjugate() * T_a).real) /\
                np.sum((x_k.conjugate() * lhs[:, 0]).real)
            x_k_n = x_k + y + T_adj
            x_k_n /= norm(x_k_n)
        else:
            x_k_n = (x_k + y) / norm(x_k + y)

        # x_k_n = (x_k + y)/(np.linalg.norm(x_k + y))

        #  update residual and lbd
        R = norm(x_k-x_k_n)
        x_k = x_k_n
        if do_chebyshev:
            T_x_m_3 = symmetric_tv_mode_product(T, x_k, m-3)
            T_x_m_2 = np.tensordot(T_x_m_3, x_k, axes=1)
        else:
            T_x_m_2 = symmetric_tv_mode_product(T, x_k, m-2)
        T_x_m_1 = T_x_m_2 @ x_k

        lbd = (x_k.conjugate().T @ T_x_m_1).real
        # print('ctr=%d lbd=%f' % (ctr, lbd))
        ctr += 1
    x = x_k
    err = norm(symmetric_tv_mode_product(
        T, x, m-1) - lbd * x)
    if ctr < max_itr:
        converge = True

    return x, lbd, ctr, converge, err



def complex_eigen_cnt(n, m):
    if m == 2:
        return n
    return (power(m-1, n)-1) // (m-2)


def find_eig_cnt(all_eig):
    first_nan = np.where(np.isnan(all_eig.x))[0]
    if first_nan.shape[0] == 0:
        return None
    else:
        return first_nan[0]

    
def normalize_real_positive(lbd, x, m, tol):
    """ First try to make it to a real pair
    if not possible. If not then make lambda real
    return is_self_conj, is_real, new_lbd, new_x
    """
    u = sqrt((x @ x).conjugate())
    # is_self_conj = norm(x.conjugate() - u*u*x) < tol
    new_x = x * u
    is_real = norm(new_x.imag) < m*tol    
    if is_real:
        # zero eigenvalue, just insert
        if np.abs(lbd) < tol:
          return True, True, lbd, new_x
        # try to flip. if u **(m-2) > 0 use it:
        lbd_factor = u**(m-2)        
        if np.abs(lbd_factor*lbd_factor - 1) < tol:
            lbd_factor = lbd_factor.real
            if lbd * lbd_factor > 0:
                return True, True, lbd * lbd_factor, new_x
            elif m % 2 == 1:
                return True, True, -lbd * lbd_factor, -new_x
            else:
                return True, True, lbd * lbd_factor, new_x

    is_self_conj = np.abs((x@x).conjugate()**(m-2)-1) < tol                
    if lbd < 0:
        return is_self_conj, False, -lbd, x * exp(pi/(m-2)*1j)
    else:
        return is_self_conj, False, lbd, x


def _insert_eigen(all_eig, x, lbd, eig_cnt, m, tol, disc):
    """
    force eigen values to be positive if possible
    if x is not similar to a vector in all_eig.x
    then:
       insert pair x, conj(x) if x is not self conjugate
       otherwise insert x
    all_eig has a structure: lbd, x, is_self_conj, is_real
    """
    is_self_conj, is_real, norm_lbd, norm_x = normalize_real_positive(
        lbd, x, m, tol)

    if is_self_conj:
        good_x = [norm_x]
    else:
        good_x = [norm_x, norm_x.conjugate()]
    nct = 0
    for xx in good_x:
      factors = all_eig.x[:eig_cnt+nct, :] @ xx.conjugate()
      fidx = np.where(np.abs(factors ** (m-2) - 1) < disc)[0]
      if fidx.shape[0] == 0:
        all_eig.lbd[eig_cnt+nct] = norm_lbd
        all_eig.x[eig_cnt+nct] = xx
        all_eig.is_self_conj[eig_cnt+nct] = is_self_conj
        all_eig.is_real[eig_cnt+nct] = is_real
        nct += 1

      if False and fidx.shape[0]:
        all_diffs = all_eig.x[:eig_cnt, :][fidx, :] -\
                    factors[fidx][:, None] * norm_x[None, :]
        all_diff_n = np.sqrt(np.sum(all_diffs*all_diffs.conj(), axis=1))
        # if np.where(np.sum(np.abs(all_diffs), axis=1) < disc * x.shape[0])[0].shape[0]:
        if np.where(all_diff_n < disc * x.shape[0])[0].shape[0]:
            return eig_cnt
    return eig_cnt + nct


def find_all_unitary_eigenpair(
        all_eig, eig_cnt, A, max_itr, max_test=int(1e6), tol=1e-10, disc=1e-6):
    """ output is the table of results
     2n*+2 columns: lbd, is self conjugate, x_real, x_imag
    This is the raw version, since the output vector x
    is not yet normalized to be real when possible
    """
    n = A.shape[0]
    m = len(A.shape)
    n_eig = complex_eigen_cnt(n, m)
    if all_eig is None:
        all_eig = SimpleNamespace(
            lbd=np.full((n_eig), np.nan, dtype=float),
            x=np.full((n_eig, n), np.nan, dtype=complex),
            is_self_conj=zeros((n_eig), dtype=bool),
            is_real=zeros((n_eig), dtype=bool))
        eig_cnt = 0
    elif eig_cnt is None:
        eig_cnt = find_eig_cnt(all_eig)
        if eig_cnt is None:
            return all_eig

    for jj in range(max_test):
        x0r = np.random.randn(2*n)
        x0r /= norm(x0r)
        x0 = x0r[:n] + x0r[n:] * 1.j
        # if there are odd numbers left,
        # try to find a real root
        draw = np.random.uniform(0, 1, 1)
        # 50% try real root
        if (draw < .5) and ((n_eig - eig_cnt) % 2 == 1):
            try:
                x_r, lbd, ctr, converge, err = schur_form_rayleigh_chebyshev(
                    A, max_itr, tol, x_init=x0.real, do_chebyshev=False)
                x = x_r + 1j * zeros((x_r.shape[0]))
            except Exception as e:
                print(e)
                continue
        else:
            try:
                x, lbd, ctr, converge, err =\
                    schur_form_rayleigh_chebyshev_unitary(
                        A, max_itr, tol, x_init=x0, do_chebyshev=False)
            except Exception as e:
                print(e)
                continue
        old_eig = eig_cnt
        # if converge and (err < tol):
        if err < tol:
            eig_cnt = _insert_eigen(all_eig, x, lbd, eig_cnt, m, tol, disc)
        if eig_cnt == n_eig:
            break
        elif (eig_cnt > old_eig) and (eig_cnt % 5 == 0):
            print('Found %d eigenpairs' % eig_cnt)
    print('Found %d eigenpairs' % eig_cnt)
    return all_eig, jj
    
def time_find_all_unitary(A, tol, disc, max_itr, save_file=None, max_test=1e6):
    # find from begining
    t_start = process_time()
    all_eig, n_runs = find_all_unitary_eigenpair(
        all_eig=None, eig_cnt=None, A=A, max_itr=max_itr, max_test=10, tol=tol)

    # continue finding more pairs
    all_eig, n_runs = find_all_unitary_eigenpair(
        all_eig, eig_cnt=None, A=A, max_itr=max_itr,
        max_test=int(max_test), tol=tol, disc=disc)

    t_end = process_time()
    tot_time = t_end - t_start
    print('tot time %f avg=%f' % (tot_time, tot_time / all_eig.x.shape[0]))

    
    if save_file is not None:
      np.savez_compressed('%s_%d_%d.npz' % (
          save_file, n, m), A=A, lbd=all_eig.lbd,
                        x=all_eig.x, is_real=all_eig.is_real,
                        is_self_conj=all_eig.is_self_conj)
    return A, all_eig


some codes to beautify the outputs - and verify the outputs

In [4]:
def check_eig(TT, all_eig):
  max_run = all_eig.lbd.shape[0]
  diff0 = np.empty(max_run)
  diff1 = np.empty(max_run)
  for i in range(max_run):
    Tnew = symmetric_tv_mode_product(TT, all_eig.x[i, :], m-1)
    lbdnew = np.sum(all_eig.x[i, :].conjugate()*Tnew).real
    diff0[i] = lbdnew - all_eig.lbd[i]
    diff1[i] = norm(Tnew - lbdnew*all_eig.x[i, :])
  print('check lbd %f' % np.max(np.abs(diff0)))
  print('check equation %f' % np.max(np.abs(diff1)))

  diff3 = np.empty(max_run-1)
  for i in range(max_run-1):
      factors = np.sum(all_eig.x[i, :].conj()*all_eig.x[i+1, :])
      diff3[i] = np.abs(factors**(m-2)-1)
  # print(np.argsort(diff3))
  print("check uniqueness")
  print(np.sort(diff3[np.where(diff3< 1e-1)]))

def display_one(comb, i):
  return (comb.x)[i, :], (comb.lbd)[i], (comb.is_real)[i], (comb.is_self_conj)[i]

def display_all_real(comb)  :
  return pd.DataFrame(
      index=np.where(comb.is_real)[0],
       data=np.concatenate([comb.lbd[np.where(comb.is_real)[0]].real[:, None], comb.x[np.where(comb.is_real)[0], :].real], axis=1),
       columns =['lbd']+ [str(i) for i in range(comb.x.shape[1])])

def display_all_complex(comb)  :
  good = np.where(np.logical_and(np.isnan(comb.lbd) == False , comb.is_real==False))[0]
  return pd.DataFrame(
      index=good,
       data=np.concatenate([comb.lbd[good].real[:, None], comb.x[good, :]], axis=1),
       columns =['lbd']+ [str(i) for i in range(comb.x.shape[1])])
    


# Example 4.2 of [Cui et. al]
CUI , C.-F., DAI , Y.-H. & NIE , J. (2014) All real eigenvalues of symmetric tensors. SIAM Journal on MatrixAnalysis and Applications, 35, 1582–1601.

According to the paper, it took 400 seconds (in a Windows machine around 10 years ago) to find these real eigenpairs.

It takes 4 seconds on colab using our algorithm to find all 85 complex eigen pairs

In [5]:
import sympy as sp
n = 4
m = 5

def rand4():
  X = np.random.randn(4)
  return X/norm(X)

w10, w11, w12, w13 = rand4()
w20, w21, w22, w23 = rand4()
w30, w31, w32, w33 = rand4()
x0, x1, x2, x3 = sp.symbols('x0 x1 x2 x3')
w1 = sp.Matrix([[w10, w11, w12, w13]]).T
w2 = sp.Matrix([[w20, w21, w22, w23]]).T
w3 = sp.Matrix([[w30, w31, w32, w33]]).T
P = (sp.eye(4) - 2*w1*w1.T)*(sp.eye(4) - 2*w2*w2.T)*(sp.eye(4) - 2*w3*w3.T)
XX = sp.Matrix([[x0, x1, x2, x3]]).T
PX = P*XX

AP = sp.expand(PX[0]**m + 2*PX[1]**m - 3*PX[2]**m - 4*PX[3]**m)
TT = utils.generate_symmetric_tensor_from_poly(XX, AP)
X = np.random.randn(n)
print("verifying that the polynomial function and the tensor evaluates to the same number")
print(AP.subs([(XX[i], X[i]) for i in range(n)]))
print(symmetric_tv_mode_product(TT, X, m))


verifying that the polynomial function and the tensor evaluates to the same number
-2.25046689829118
-2.250466898291175


Now run the code. The first table shows the eigenvalue and the vector next to it. The next one shows complex pairs. To analyze further the user can examine all_eig, which has the fields listed below

In [6]:

n_eig = complex_eigen_cnt(n, m)
print("expecting  %d eigenvalues" % n_eig)
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))
_, all_eig = time_find_all_unitary(TT, tol=1e-6, disc=1e-3, max_itr=200)
display(display_all_real(all_eig).sort_values(by='lbd', ascending=False))
display(display_all_complex(all_eig))
check_eig(TT, all_eig)

expecting  85 eigenvalues
Found 5 eigenpairs
Found 15 eigenpairs
Found 17 eigenpairs
Found 25 eigenpairs
Found 35 eigenpairs
Found 45 eigenpairs
Found 65 eigenpairs
Found 75 eigenpairs
Found 85 eigenpairs
tot time 2.892341 avg=0.034028


,lbd,0,1,2,3
56,4.000000,-0.423196,-0.052074,0.282328,0.859351
75,3.000000,-0.405004,0.887954,-0.203190,-0.078885
78,2.000000,0.103574,0.239652,0.934615,-0.241526
62,1.216340,-0.584339,0.622189,0.039465,0.519492
25,1.000000,0.803834,0.389092,-0.074163,0.443799
38,0.961091,-0.181966,0.155338,0.907574,0.345063
44,0.854277,-0.188449,0.764667,0.570186,-0.233793
76,0.605725,0.454560,0.301457,0.087734,0.833547
43,0.554980,0.429811,0.825703,-0.176723,0.319760
61,0.540241,-0.378883,0.629632,0.634271,0.240231


,lbd,0,1,2,3
1,0.854277+0.000000j,-0.305453-0.067552j,0.493943-0.156303j,-0.485611-0.609564j,0.039049+0.157526j
2,0.854277+0.000000j,-0.305453+0.067552j,0.493943+0.156303j,-0.485611+0.609564j,0.039049-0.157526j
3,0.961091+0.000000j,-0.303656+0.070258j,-0.126229+0.162563j,-0.190510+0.633979j,0.628833-0.163835j
4,0.961091+0.000000j,-0.303656-0.070258j,-0.126229-0.162563j,-0.190510-0.633979j,0.628833+0.163835j
5,0.326105+0.000000j,-0.208293+0.597551j,-0.411259-0.248448j,-0.181287-0.402415j,-0.067938+0.411422j
...,...,...,...,...,...
80,0.251839+0.000000j,0.479847+0.153576j,0.150716-0.336709j,0.578416+0.077049j,0.518339+0.029913j
81,0.540241+0.000000j,0.289871-0.010028j,-0.082440+0.457392j,0.589104-0.224817j,-0.354309-0.420415j
82,0.540241+0.000000j,0.289871+0.010028j,-0.082440-0.457392j,0.589104+0.224817j,-0.354309+0.420415j
83,0.251839+0.000000j,-0.372925+0.630373j,0.216241+0.334760j,-0.355935+0.267862j,-0.285075-0.158194j


check lbd 0.000000
check equation 0.000000
check uniqueness
[]


# Example 4.3 from Cui, Dai et. al
$T = 2x_0^4 + 3x_1^4 + 5x_2^4 + 4ax_0^2x_1x_2$

It finds all 13 pairs instantly, less than a second. Setting up the tensor then run the eigenpair timing code. Take $a=3$ below, but we can just change $a$ then rerun

In [7]:
n = 3
m = 4
x0, x1, x2 = sp.symbols('x0 x1 x2')
XX = sp.Matrix([x0, x1, x2])
a = 3
P = 2*x0**4 + 3*x1**4 + 5*x2**4 + 4*a*x0**2*x1*x2
T = utils.generate_symmetric_tensor_from_poly(XX, P)
X = np.random.randn(3)
print(P.subs([(XX[i], X[i]) for i in range(3)]))
print(symmetric_tv_mode_product(T, X, 4))


14.5857798021153
14.585779802115344


In [8]:

n_eig = complex_eigen_cnt(n, m)
print("expecting  %d eigenvalues" % n_eig)
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))
_, all_eig = time_find_all_unitary(T, tol=1e-6, disc=1e-3, max_itr=200)


display(display_all_real(all_eig).sort_values(by='lbd', ascending=False))
check_eig(T, all_eig)

expecting  13 eigenvalues
Found 5 eigenpairs
Found 8 eigenpairs
Found 10 eigenpairs
Found 13 eigenpairs
tot time 0.121253 avg=0.009327


,lbd,0,1,2
7,5.000000,8.502835e-24,1.812689e-23,1.000000e+00
3,3.000000,3.337292e-14,-1.000000e+00,1.732747e-14
11,2.214715,4.875358e-01,7.928607e-01,3.656237e-01
10,2.214715,-4.875358e-01,7.928607e-01,3.656237e-01
9,2.000000,1.000000e+00,-1.588836e-13,-1.588681e-13
4,1.875000,-1.514322e-22,-7.905694e-01,6.123724e-01
2,1.875000,6.833787e-13,-7.905694e-01,-6.123724e-01
12,-0.512644,7.042264e-01,5.266924e-01,-4.760885e-01
8,-0.512644,7.042264e-01,-5.266924e-01,4.760885e-01


check lbd 0.000000
check equation 0.000000
check uniqueness
[0.09108039 0.09108039]


# Example 4.6 from [Cui et al.], or 9.1 from [Qi et al.]
[L. Qi, F. Wang, and Y. Wang.] Z-eigenvalue methods for a global polynomial optimization problem, Mathematical Programming, 118 (2009), pp. 301–316.

There is a typo in [Cui et al.]. According to [Qi et al.], the polynomial is
$\sum_{i=0}^5(i+1)x_i^3 + 30\sum_{i=0}^4x_i^2x_{i+1} $
Numerically, the result confirm the coefficients in [Qi, Wang et al.]. It takes 4.7 seconds in our algorithm, and took 10870 seconds in the algorithm in Cui, Dai et Al.

In [9]:
n = 6
m = 3
x0, x1, x2, x3, x4, x5 = sp.symbols('x0 x1 x2 x3 x4 x5')
XX = sp.Matrix([x0, x1, x2, x3, x4, x5])

P = 0
for i in range(n):
  P = P + (i+1)*XX[i]**3
for i in range(n-1):
    P = P + 30*XX[i]**2*XX[i+1]
T = utils.generate_symmetric_tensor_from_poly(XX, P)
X = np.random.randn(n)
print(P.subs([(XX[i], X[i]) for i in range(n)]))
print(symmetric_tv_mode_product(T, X, m))


-0.482324201736757
-0.48232420173673773


In [10]:
n_eig = complex_eigen_cnt(n, m)
print("expecting  %d eigenvalues" % n_eig)
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))
_, all_eig = time_find_all_unitary(T, tol=1e-6, disc=1e-3, max_itr=200)
display(display_all_real(all_eig).sort_values(by='lbd', ascending=False))
check_eig(T, all_eig)

expecting  63 eigenvalues
Found 5 eigenpairs
Found 11 eigenpairs
Found 15 eigenpairs
Found 20 eigenpairs
Found 25 eigenpairs
Found 40 eigenpairs
Found 45 eigenpairs
Found 55 eigenpairs
Found 63 eigenpairs
tot time 4.527759 avg=0.071869


,lbd,0,1,2,3,4,5
4,16.234514,-8.065373e-31,1.152632e-18,-3.861466e-15,6.576900e-01,6.801877e-01,0.323711
46,15.455152,9.027797e-36,-1.296234e-27,-3.217333e-19,-2.059282e-01,8.139433e-01,0.543222
2,15.429818,-1.970576e-46,7.503941e-27,-1.256202e-21,5.438581e-20,8.249106e-01,0.565263
32,10.971097,-4.176686e-14,-1.165408e-13,-7.238450e-14,-6.611273e-14,-6.922834e-01,0.721626
61,8.734734,3.666544e-01,4.184040e-01,2.342437e-01,2.792536e-02,-5.512904e-01,0.575267
38,8.659601,-2.973235e-01,4.478462e-01,2.894994e-01,4.315336e-02,-5.467213e-01,0.571363
3,8.597884,9.309336e-23,4.759539e-01,3.822988e-01,7.627314e-02,-5.434459e-01,0.571108
62,8.188851,3.019846e-17,-4.543818e-01,4.548807e-01,1.142687e-01,-5.188059e-01,0.551728
16,7.216542,-2.222384e-17,3.232570e-14,6.395782e-01,2.648904e-01,-4.642822e-01,0.552463
39,6.000000,-1.234763e-24,-5.711828e-23,1.001179e-22,3.455499e-23,-1.074830e-23,1.000000


check lbd 0.000000
check equation 0.000000
check uniqueness
[0.00814615 0.03360424 0.03885793 0.04627523 0.05772557 0.07988169]


# Example 4.7 of [Cui, Dai et al.] 

There are 364 eigenpairs, it took us around 300 seconds to find them, versus Cui, Dai et al 280 seconds for the real pairs(on Windows - many years ago). However, we found all eigenvectors for the eigenvector -4.5 (due to a rank condition in that paper, only one eigenvector was found there).

In [11]:
import sympy as sp
x0, x1, x2, x3, x4, x5 = sp.symbols('x0 x1 x2 x3 x4 x5')
XX = sp.Matrix([x0, x1, x2, x3, x4, x5])
P = 0
for i in range(5):
  for j in range(i, 6):
    P = P - (XX[i]-XX[j])**4
TT = utils.generate_symmetric_tensor_from_poly(XX, P)

X = np.random.randn(6)
print(P.subs([(XX[i], X[i]) for i in range(6)]))
print(symmetric_tv_mode_product(TT, X, 4))


-339.231616145662
-339.231616145662


In [12]:
n = 6
m = 4
np.random.seed(0)
n_eig = complex_eigen_cnt(n, m)
print("expecting  %d eigenvalues" % n_eig)
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))
# _, all_eig = time_find_all_unitary(TT, 1e-6, 1e-3, max_itr)
_, all_eig = time_find_all_unitary(TT, tol=1e-6, disc=5e-3, max_itr=200)
# all_eig, n_runs = find_all_unitary_eigenpair(all_eig, eig_cnt=None, A=TT, max_itr=200, max_test=int(1e6), tol=1e-6, disc=5e-3)
check_eig(TT, all_eig)
display_all_real(all_eig).sort_values(by='lbd')

expecting  364 eigenvalues
Found 5 eigenpairs
Found 6 eigenpairs
Found 10 eigenpairs
Found 15 eigenpairs
Found 20 eigenpairs
Found 30 eigenpairs
Found 40 eigenpairs
Found 50 eigenpairs
Found 55 eigenpairs
Found 60 eigenpairs
Found 70 eigenpairs
Found 80 eigenpairs
Found 90 eigenpairs
Found 95 eigenpairs
Singular matrix
Found 100 eigenpairs
Found 110 eigenpairs
Found 120 eigenpairs
Found 130 eigenpairs
Found 135 eigenpairs
Found 150 eigenpairs
Found 170 eigenpairs
Found 180 eigenpairs
Found 190 eigenpairs
Found 200 eigenpairs
Found 210 eigenpairs
Found 220 eigenpairs
Found 230 eigenpairs
Found 235 eigenpairs
Found 250 eigenpairs
Found 260 eigenpairs
Found 280 eigenpairs
Found 290 eigenpairs
Found 295 eigenpairs
Found 300 eigenpairs
Found 310 eigenpairs
Found 315 eigenpairs
Found 320 eigenpairs
Found 330 eigenpairs
Found 340 eigenpairs
Found 350 eigenpairs
Found 360 eigenpairs
Found 364 eigenpairs
tot time 266.400959 avg=0.731871
check lbd 0.000000
check equation 0.000000
check uniquenes

,lbd,0,1,2,3,4,5
123,-7.200000e+00,1.825742e-01,-9.128709e-01,1.825742e-01,1.825742e-01,1.825742e-01,1.825742e-01
92,-7.200000e+00,9.128709e-01,-1.825742e-01,-1.825742e-01,-1.825742e-01,-1.825742e-01,-1.825742e-01
237,-7.200000e+00,-1.825742e-01,-1.825742e-01,9.128709e-01,-1.825742e-01,-1.825742e-01,-1.825742e-01
141,-7.200000e+00,-1.825742e-01,-1.825742e-01,-1.825742e-01,9.128709e-01,-1.825742e-01,-1.825742e-01
317,-7.200000e+00,1.825742e-01,1.825742e-01,1.825742e-01,1.825742e-01,-9.128709e-01,1.825742e-01
113,-6.000000e+00,-1.149270e-16,-1.316276e-16,-7.071068e-01,-8.245682e-17,-5.224143e-17,7.071068e-01
331,-6.000000e+00,1.229662e-16,7.071068e-01,7.252073e-17,5.422671e-17,1.074485e-16,-7.071068e-01
37,-6.000000e+00,1.108094e-16,1.395419e-16,1.048711e-16,7.071068e-01,1.376140e-16,-7.071068e-01
161,-6.000000e+00,-7.071068e-01,7.071068e-01,2.826723e-17,-4.372203e-17,2.847234e-17,3.207476e-17
177,-6.000000e+00,5.192750e-17,5.555375e-17,-7.071068e-01,5.310493e-17,7.071068e-01,5.118612e-17


We show $\lambda = -9/2$ has infinitely many eigenvectors of the form
$$\begin{bmatrix}x_0\\x_0\\x_2\\x_2\\-x_0-x_2\\-x_0-x_2\end{bmatrix}$$
with $x_2 =\frac{-x_0 \pm \sqrt{1-3x_0^2}}{2}$
We got this solution by inspecting the eigenvectors of $\lambda=-\frac{9}{2}$ found numerically above

In [13]:
import sympy as sp
x0, x1, x2, x3, x4, x5, z = sp.symbols('x0 x1 x2 x3 x4 x5 z')
XX = sp.Matrix([x0, x1, x2, x3, x4, x5])
P = 0
for i in range(5):
  for j in range(i, 6):
    P = P - (XX[i]-XX[j])**4
gr = sp.Matrix([sp.diff(P, XX[i]) for i in range(6)])/sp.Integer(4)
grs = gr.subs([(x1, x0), (x3, x2), (x5, x4)])

grs2a = sp.expand(grs.subs(x4, -x0  -x2))
x2s = (-x0 - sp.sqrt(1-3*x0*x0))/2
Xnew = sp.Matrix([x0, x0, x2s, x2s, -x0-x2s, -x0-x2s])


print("CHECK NORM IS 1 and eigenvector")
grs2b = sp.simplify(sp.expand(grs2a.subs(x2, x2s)))
display(sp.simplify(Xnew.dot(Xnew)))

display(sp.simplify(grs2b + sp.Integer(9)/sp.Integer(2)* Xnew))



CHECK NORM IS 1 and eigenvector


1

Matrix([
[0],
[0],
[0],
[0],
[0],
[0]])

An orthogonal complement of Xnew using symbolic Householder

In [14]:
n = 6
m = 4

I2 = sp.eye(n)[:, 1:]
v = Xnew  - sp.Matrix([1] + (n-1)*[0])
v = v / sp.sqrt(v.dot(v))
Q = I2 - 2*v.reshape(n, 1)*v.reshape(1, n)*I2
display(Q)
display(sp.simplify(sp.expand(Xnew.T*Q)))
display(sp.simplify(sp.expand(Q.T*Q)))



Matrix([
[                     -2*x0*(x0 - 1)/(x0**2 + 2*(-x0/2 - sqrt(1 - 3*x0**2)/2)**2 + 2*(-x0/2 + sqrt(1 - 3*x0**2)/2)**2 + (x0 - 1)**2),                      -2*(-x0/2 - sqrt(1 - 3*x0**2)/2)*(x0 - 1)/(x0**2 + 2*(-x0/2 - sqrt(1 - 3*x0**2)/2)**2 + 2*(-x0/2 + sqrt(1 - 3*x0**2)/2)**2 + (x0 - 1)**2),                      -2*(-x0/2 - sqrt(1 - 3*x0**2)/2)*(x0 - 1)/(x0**2 + 2*(-x0/2 - sqrt(1 - 3*x0**2)/2)**2 + 2*(-x0/2 + sqrt(1 - 3*x0**2)/2)**2 + (x0 - 1)**2),                      -2*(-x0/2 + sqrt(1 - 3*x0**2)/2)*(x0 - 1)/(x0**2 + 2*(-x0/2 - sqrt(1 - 3*x0**2)/2)**2 + 2*(-x0/2 + sqrt(1 - 3*x0**2)/2)**2 + (x0 - 1)**2),                      -2*(-x0/2 + sqrt(1 - 3*x0**2)/2)*(x0 - 1)/(x0**2 + 2*(-x0/2 - sqrt(1 - 3*x0**2)/2)**2 + 2*(-x0/2 + sqrt(1 - 3*x0**2)/2)**2 + (x0 - 1)**2)],
[                       -2*x0**2/(x0**2 + 2*(-x0/2 - sqrt(1 - 3*x0**2)/2)**2 + 2*(-x0/2 + sqrt(1 - 3*x0**2)/2)**2 + (x0 - 1)**2) + 1,                            -2*x0*(-x0/2 - sqrt(1 - 3*x0**2)/2)/(x0**2 + 2*(-x0/2 -

Matrix([[0, 0, 0, 0, 0]])

Matrix([
[1, 0, 0, 0, 0],
[0, 1, 0, 0, 0],
[0, 0, 1, 0, 0],
[0, 0, 0, 1, 0],
[0, 0, 0, 0, 1]])

Show the $F^{\natural}$ does not exists  at these eigenpairs by showing $Q^T((m-1)\cT(I, I, X^{[m-2]}) - \lambda I)Q$ is not invertible.

In [15]:
hess = sp.zeros(6, 6)
for i in range(6):
  hess[:, i] = sp.diff(gr, XX[i])
hessp = Q.T*(hess - P*sp.eye(n)).subs([(XX[i], Xnew[i]) for i in range(6)])*Q
sp.simplify(hessp.det())

0

In [16]:
display_all_complex(all_eig)

,lbd,0,1,2,3,4,5
1,1.488386e-17+0.000000e+00j,0.035137+0.406732j,0.035141+0.406736j,0.035134+0.406732j,0.035142+0.406735j,0.035139+0.406733j,0.035137+0.406732j
2,1.488386e-17+0.000000e+00j,0.035137-0.406732j,0.035141-0.406736j,0.035134-0.406732j,0.035142-0.406735j,0.035139-0.406733j,0.035137-0.406732j
6,3.021523e-17+0.000000e+00j,-0.000783-0.408248j,-0.000787-0.408247j,-0.000784-0.408247j,-0.000785-0.408248j,-0.000787-0.408248j,-0.000784-0.408247j
7,3.021523e-17+0.000000e+00j,-0.000783+0.408248j,-0.000787+0.408247j,-0.000784+0.408247j,-0.000785+0.408248j,-0.000787+0.408248j,-0.000784+0.408247j
8,3.977022e-18+0.000000e+00j,0.215126+0.346968j,0.215127+0.346968j,0.215127+0.346969j,0.215129+0.346968j,0.215127+0.346970j,0.215122+0.346968j
...,...,...,...,...,...,...,...
359,1.069348e-17+0.000000e+00j,0.165171+0.373338j,0.165178+0.373340j,0.165173+0.373342j,0.165178+0.373340j,0.165184+0.373340j,0.165180+0.373342j
360,2.017220e-17+0.000000e+00j,-0.405764-0.044957j,-0.405761-0.044948j,-0.405763-0.044952j,-0.405769-0.044957j,-0.405772-0.044960j,-0.405768-0.044923j
361,2.017220e-17+0.000000e+00j,-0.405764+0.044957j,-0.405761+0.044948j,-0.405763+0.044952j,-0.405769+0.044957j,-0.405772+0.044960j,-0.405768+0.044923j
362,4.483769e+00+0.000000e+00j,-0.011241+0.512631j,-0.013250-0.485349j,0.024491-0.027281j,-0.013250-0.485349j,0.024491-0.027281j,-0.011241+0.512631j


# Motzkin polynomial CARTWRIGHT , D. & STURMFELS , B. (2013), example 5.9

The paper convention using a tensor 6 times our tensor. So eigenvalues there dividing by 6 will match ours.

In this case, we find only 23 eigenpairs.
* The "missing" 8 are actually eigenpairs with multiplicity. A small perturbation shows there are 14 eigenvectors with eigenvalue 0, but collapse to only 6.
* Comparing with Cartwright- Stumfels 2013, counting only 25 pairs, their "missing pairs" are 6 complex pairs of eigenvalues 
  * 1/12 complex eigen vectors $(x_0, -\bar{x_0}, 0)$ with $x_0^4=-\frac{1}{4}$ (2 equivalent pairs)
  * 3/16, eigenvectors $(0, 1, 0), (1, 0, 0), (x_0, x_1, \frac{\sqrt{2}}{2})$, $x_0^2 = x_1^2 =-\frac{1}{4}$ (4 equivalent pairs)
* The cited paper uses the normalization $X^TX = 1$, these vectors satisfy $X^TX =0$.

Gradient:
$$6T(I, X^{[5]}) = \left[\begin{matrix}4 x_{0}^{3} x_{1}^{2} + 2 x_{0} x_{1}^{4} - 6 x_{0} x_{1}^{2} x_{2}^{2}\\2 x_{0}^{4} x_{1} + 4 x_{0}^{2} x_{1}^{3} - 6 x_{0}^{2} x_{1} x_{2}^{2}\\- 6 x_{0}^{2} x_{1}^{2} x_{2} + 6 x_{2}^{5}\end{matrix}\right]$$

CARTWRIGHT , D. & STURMFELS , B. (2013) The number of eigenvalues of a tensor. Linear Algebra and its
Applications, 438, 942 – 952. Tensors and Multilinear Algebra.

In [17]:
n = 3
m = 6
x0, x1, x2 = sp.symbols('x0 x1 x2')
XX = sp.Matrix([x0, x1, x2])

P = x0**4*x1**2 + x0**2*x1**4 + x2**6 - 3*x0**2*x1**2*x2**2
T = utils.generate_symmetric_tensor_from_poly(XX, P)
X = np.random.randn(3)
print(P.subs([(XX[i], X[i]) for i in range(n)]))
print(symmetric_tv_mode_product(T, X, m))


7.90959944425763
7.909599444257626


In [18]:
np.random.seed(0)
n_eig = complex_eigen_cnt(n, m)
print("expecting  %d eigenvalues" % n_eig)
all_eig = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))

_, all_eig = time_find_all_unitary(T, tol=1e-6, disc=5e-3, max_itr=200, max_test=2e3)
check_eig(T, all_eig)
display_all_real(all_eig).sort_values(by='lbd')

expecting  31 eigenvalues
Found 5 eigenpairs
Found 5 eigenpairs
Found 10 eigenpairs
Singular matrix
Singular matrix
Found 20 eigenpairs


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: RuntimeWarning: invalid value encountered in true_divide


Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in double_scalars


Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Singular matrix
Found 23 eigenpairs
tot time 63.177567 avg=2.037986
check lbd nan
check equation nan
check uniqueness
[]


,lbd,0,1,2
3,-1.347084e-17,5.773503e-01,-5.773503e-01,-5.773503e-01
1,-4.641582e-34,-3.847635e-295,-1.000000e+00,2.308716e-06
2,2.818596e-33,1.000000e+00,1.056484e-272,3.758421e-06
19,8.595497e-21,-5.773503e-01,-5.773503e-01,5.773503e-01
4,2.003086e-18,-5.773503e-01,5.773503e-01,-5.773503e-01
22,4.006172e-18,-5.773503e-01,-5.773503e-01,-5.773503e-01
16,1.562500e-02,-8.253401e-01,2.623238e-01,5.000000e-01
11,1.562500e-02,-2.623238e-01,8.253401e-01,-5.000000e-01
0,1.562500e-02,8.253401e-01,2.623238e-01,5.000000e-01
7,1.562500e-02,2.623238e-01,8.253401e-01,-5.000000e-01


In [19]:
display_all_complex(all_eig)

,lbd,0,1,2
5,0.083333+0.000000j,-0.5+0.5j,0.5+0.5j,-5.591232e-22+3.705890e-22j
6,0.083333+0.000000j,-0.5-0.5j,0.5-0.5j,-5.591232e-22-3.705890e-22j
14,0.187500+0.000000j,-0.0-0.5j,0.0+0.5j,-7.071068e-01+5.551115e-17j
15,0.187500+0.000000j,-0.0+0.5j,0.0-0.5j,-7.071068e-01-5.551115e-17j
17,0.187500+0.000000j,-0.5-0.0j,-0.5-0.0j,6.555831e-17+7.071068e-01j
18,0.187500+0.000000j,-0.5+0.0j,-0.5+0.0j,6.555831e-17-7.071068e-01j


perturb, we see the there are actually 14 eigenvector of eigenvalue zero, they just collapse to 6 above

In [20]:
print("test1")
xx = np.array([.5j, .5j, np.sqrt(2)/2])
print(np.sum(xx.conjugate()*symmetric_tv_mode_product(T, xx, 5)), 3/16)

print(symmetric_tv_mode_product(T, xx, 5 ) - 3/16*xx)
print("test2")
xx = np.array([.5 + .5j, -.5 + .5j , 0])
print(np.sum(xx.conjugate()*symmetric_tv_mode_product(T, xx, 5)), 1/12)
print(symmetric_tv_mode_product(T, xx, 5 ) - 1/12*xx)

test1
(0.1875+0j) 0.1875
[0.+0.j 0.+0.j 0.+0.j]
test2
(0.08333333333333334+0j) 0.08333333333333333
[ 6.9388939e-18+6.9388939e-18j -6.9388939e-18+6.9388939e-18j
  0.0000000e+00+0.0000000e+00j]


In [21]:
Ta = T.copy()
for i in range(n):
  Ta[m*[i]] += (i+1)*1e-5

all_eig2 = SimpleNamespace(
    lbd=np.full((n_eig), np.nan, dtype=complex),
    x=np.full((n_eig, n), np.nan, dtype=complex),
    is_self_conj=zeros((n_eig), dtype=bool),
    is_real=zeros((n_eig), dtype=bool))

_, all_eig2 = time_find_all_unitary(Ta, tol=1e-6, disc=5e-3, max_itr=200, max_test=2e3)
check_eig(Ta, all_eig2)
display_all_real(all_eig2).sort_values(by='lbd')

Found 5 eigenpairs
Found 13 eigenpairs
Found 15 eigenpairs
Found 20 eigenpairs
Found 25 eigenpairs
Found 30 eigenpairs
Found 31 eigenpairs
tot time 1.163512 avg=0.037533
check lbd 0.000001
check equation 0.000000
check uniqueness
[]


,lbd,0,1,2
15,-7.406198e-07,0.577338,-0.577364,5.773480e-01
27,-2.221975e-11,-0.577353,-0.577344,5.773532e-01
19,1.481183e-06,0.577364,-0.577338,-5.773487e-01
2,5.402014e-06,0.993842,-0.000034,-1.108045e-01
29,1.075728e-05,0.000017,-0.993764,1.115013e-01
18,5.404407e-04,-0.578528,-0.576406,-5.771151e-01
12,1.562489e-02,-0.825338,-0.262321,5.000055e-01
26,1.562500e-02,0.825340,-0.262324,-5.000000e-01
17,1.562500e-02,0.262324,-0.825340,5.000000e-01
5,1.562529e-02,0.262312,0.825343,-5.000012e-01


And 8 eigenvectors of small eigenvalues in the complex part

In [22]:
display_all_complex(all_eig2)
# all_eig.x[14]

,lbd,0,1,2
0,0.000016+0.000000j,-0.985789-0.098879j,0.000094-0.000035j,-0.111919+0.076924j
1,0.000016+0.000000j,-0.985789+0.098879j,0.000094+0.000035j,-0.111919-0.076924j
3,0.000008+0.000000j,0.982799-0.141389j,-0.000043-0.000018j,-0.032650+0.114231j
4,0.000008+0.000000j,0.982799+0.141389j,-0.000043+0.000018j,-0.032650-0.114231j
7,0.000016+0.000000j,-0.000009-0.000021j,-0.142265+0.982614j,0.114927-0.031958j
8,0.000016+0.000000j,-0.000009+0.000021j,-0.142265-0.982614j,0.114927+0.031958j
9,0.000032+0.000000j,-0.000017-0.000047j,-0.097297+0.986077j,0.075716+0.111587j
10,0.000032+0.000000j,-0.000017+0.000047j,-0.097297-0.986077j,0.075716-0.111587j
13,0.083348+0.000000j,-0.500022-0.499992j,0.500022-0.499962j,-0.000000-0.000090j
14,0.083348+0.000000j,-0.500022+0.499992j,0.500022+0.499962j,-0.000000+0.000090j


# Example 4.11
In these examples, we show the operator $\Pi L_{X}$ is not invertible, a small perturbation gives us information on other eigenpairs

In [23]:
n = 5
m = 3
np.random.seed(0)
neig0 = complex_eigen_cnt(n, m)
T = np.zeros((n, n, n))

def mpw(i):
  return 1 - 2*(i % 2)
mpw(6)

for i0 in range(n):
  for i1 in range(n):
    for i2 in range(n):
          T[i0, i1, i2] = mpw(i0+1)/(i0+1) + mpw(i1+1)/(i1+1) + mpw(i2+1)/(i2+1)

sch_x, sch_lbd, ctr, converge, err = schur_form_rayleigh_chebyshev(
              T, max_itr=30, delta=1e-8, x_init=np.array([1] + (n-1)*[0.]), do_chebyshev=False)

print(sch_x, sch_lbd, ctr, converge, err)

from scipy.linalg import null_space
Q = null_space(sch_x.reshape(1, -1))
hess = Q.T@((m-1)*symmetric_tv_mode_product(T, sch_x, m-2))@Q
print(np.linalg.det(hess))


[ 0.10938332 -0.44766753 -0.24032136  0.81962463 -0.24101905] -1.8674273216398812e-17 30 False 8.2401295180647e-17
4.127827387189961e-52


A heuristic fix: Add a small tensor

In [24]:
eps = 1e-3
print("expecting %d pairs" % complex_eigen_cnt(n, m))
Ta = T.copy()
for i0 in range(n):          
  # T[i0, i0, i0] += eps*np.random.uniform(1e-1, 1)
  Ta[i0, i0, i0] += eps*(i0+1)
ret = time_find_all_unitary(Ta, tol=1e-7, disc=5e-4, max_itr=200)
check_eig(Ta, ret[1])
display(display_all_real(ret[1]).sort_values(by='lbd', ascending=False))
display_all_complex(ret[1])


expecting 31 pairs
Found 5 eigenpairs
Found 9 eigenpairs
Found 15 eigenpairs
Found 20 eigenpairs
Found 25 eigenpairs
Found 30 eigenpairs
Found 31 eigenpairs
tot time 8.194670 avg=0.264344
check lbd 0.000000
check equation 0.000000
check uniqueness
[0.00031632 0.0005752  0.00212463 0.00544279 0.06996221 0.08561492
 0.09327074]


,lbd,0,1,2,3,4
27,9.976781,-0.731311,-0.137555,-0.467386,-0.236504,-0.414582
24,4.289229,-0.185810,0.715793,0.214824,0.565624,0.295016
6,0.003434,-0.283063,-0.200681,-0.225436,-0.194004,0.889462
14,0.002746,-0.247592,-0.230783,-0.217896,0.893685,-0.198205
17,0.002025,-0.257554,-0.234023,0.893695,-0.205262,-0.195132
2,0.001962,-0.358626,-0.151540,0.885488,-0.162591,-0.194675
3,0.001906,-0.380729,-0.149384,0.875113,-0.163810,-0.200184
4,0.001299,-0.275330,0.892561,-0.220937,-0.204661,-0.191910
22,0.001189,-0.419995,-0.355187,-0.315979,0.585292,0.505012
5,0.000941,-0.436677,-0.357178,0.629555,-0.284958,0.451880


,lbd,0,1,2,3,4
0,0.000105+0.000000j,0.380941-0.156904j,0.623919-0.224256j,-0.336657+0.139146j,-0.378274+0.148219j,-0.282764+0.113946j
1,0.000105+0.000000j,0.380941+0.156904j,0.623919+0.224256j,-0.336657-0.139146j,-0.378274-0.148219j,-0.282764-0.113946j
7,0.003482+0.000000j,-0.240467+0.018821j,-0.225476-0.018030j,-0.215761+0.001321j,-0.206189-0.010413j,0.895253+0.001343j
8,0.003482+0.000000j,-0.240467-0.018821j,-0.225476+0.018030j,-0.215761-0.001321j,-0.206189+0.010413j,0.895253-0.001343j
9,0.000641+0.000000j,-0.254844+0.005747j,-0.527913+0.051846j,0.538380-0.025913j,0.530140-0.033338j,-0.283615+0.022777j
10,0.000641+0.000000j,-0.254844-0.005747j,-0.527913-0.051846j,0.538380+0.025913j,0.530140+0.033338j,-0.283615-0.022777j
15,0.001929+0.000000j,0.079754-0.010218j,-0.475086+0.034601j,-0.206281+0.015423j,0.821768-0.023127j,-0.218576+0.015123j
16,0.001929+0.000000j,0.079754+0.010218j,-0.475086-0.034601j,-0.206281-0.015423j,0.821768+0.023127j,-0.218576-0.015123j
18,0.000750+0.000000j,-0.124679+0.000637j,-0.579075+0.060669j,-0.321528+0.030992j,0.587187-0.039293j,0.440491-0.027223j
19,0.000750+0.000000j,-0.124679-0.000637j,-0.579075-0.060669j,-0.321528-0.030992j,0.587187+0.039293j,0.440491+0.027223j


Now rerun the original tensor with the perturbed eigenvector found

In [25]:
x_init =np.array([-0.731364,	-0.137583,	-0.467359,	-0.236516,	-0.414504])
x_init = x_init/norm(x_init)
sch_x, sch_lbd, ctr, converge, err = schur_form_rayleigh_chebyshev(
              T, max_itr=30, delta=1e-8, x_init=x_init, do_chebyshev=False)
print(sch_x, sch_lbd, ctr, converge, err)


[-0.73128513 -0.1375412  -0.46739894 -0.23649853 -0.4146217 ] 9.977892792857288 2 True 3.4684476073050936e-15


# Example 4.12
Error is zero, but does not converge. The run oscillates between eigenvalues. Hessian is degenerate.

In [26]:
n = 5
m = 4
np.random.seed(0)
neig0 = complex_eigen_cnt(n, m)
T = np.zeros((n, n, n, n))

for i0 in range(n):
  for i1 in range(n):
    for i2 in range(n):
      for i3 in range(n):
          #T[i0, i1, i2, i3] = np.tan(i0+1) + np.tan(i1+1) + np.tan(i2+1) + np.tan(i3+1)
          T[i0, i1, i2, i3] = np.sin(i0+1 + i1+1 + i2+1 + i3+1)

sch_x, sch_lbd, ctr, converge, err = schur_form_rayleigh_chebyshev(
              T, max_itr=30, delta=1e-8, x_init=np.array([1] + (n-1)*[0.]), do_chebyshev=False)


print(sch_x, sch_lbd, ctr, converge, err)

from scipy.linalg import null_space
Q = null_space(sch_x.reshape(1, -1))
hess = Q.T@((m-1)*symmetric_tv_mode_product(T, sch_x, m-2))@Q
print(np.linalg.det(hess))


[-0.78362815  0.21983508 -0.17942712 -0.54775296 -0.0732944 ] 3.021182773080731e-17 30 False 1.4703305038227621e-16
1.102804693282504e-51


Add a small identity matrix

In [27]:
eps = 1e-3          
print("expecting %d pairs" % complex_eigen_cnt(n, m))
Ta = T.copy()
for i0 in range(n):          
  Ta[i0, i0, i0, i0] += eps*(i0+1)
ret = time_find_all_unitary(Ta, tol=1e-6, disc=1e-3, max_itr=200)
check_eig(Ta, ret[1])
display_all_real(ret[1]).sort_values(by='lbd', ascending=False)


expecting 121 pairs
Found 5 eigenpairs
Found 10 eigenpairs
Found 11 eigenpairs
Found 15 eigenpairs
Found 25 eigenpairs
Found 35 eigenpairs
Found 40 eigenpairs
Found 45 eigenpairs
Found 55 eigenpairs
Found 65 eigenpairs
Found 75 eigenpairs
Found 90 eigenpairs
Found 95 eigenpairs
Found 100 eigenpairs
Found 105 eigenpairs
Found 110 eigenpairs
Found 115 eigenpairs
Found 120 eigenpairs
Found 121 eigenpairs
tot time 51.255351 avg=0.423598
check lbd 0.000000
check equation 0.000000
check uniqueness
[0.00556308 0.01068455 0.0154853  0.01726295 0.02145898 0.022101
 0.02737957 0.03186316 0.0337832  0.03698002 0.03982465 0.04738966
 0.04831026 0.05238989 0.05240003 0.06213286 0.0624498  0.07160573
 0.07168515 0.07570882 0.07793015 0.08074271 0.08426666 0.08729331
 0.09121443 0.0954195  0.09803603]


,lbd,0,1,2,3,4
119,7.260494,0.268611,0.614981,0.395899,-0.187138,-0.598239
117,4.641793,-0.505473,0.122745,0.638252,0.566929,-0.025627
99,0.002298,-0.225010,-0.456359,-0.193096,0.186404,-0.817970
94,0.002260,0.170559,0.475383,0.268212,-0.121314,0.811336
55,0.001954,0.199879,0.546671,0.233853,-0.155665,0.763073
10,0.001772,0.419511,0.088030,-0.367759,0.811174,-0.151697
114,0.001645,-0.457927,-0.130317,0.323219,-0.790438,0.209900
89,0.001478,-0.446804,-0.111437,0.433953,-0.752022,0.184651
58,0.001453,0.186391,-0.079698,-0.416384,0.688395,-0.558250
39,0.001379,0.078419,-0.152815,-0.483409,0.635507,-0.577014


Time running a random (3, 8) tensor

In [28]:
n = 8
m = 3
np.random.seed(0)
tol = 1e-6
disc = 1e-3
max_itr = 200
print("expecting %d pairs" % complex_eigen_cnt(n, m))
T = utils.generate_symmetric_tensor(n, m)    
ret = time_find_all_unitary(T, tol, disc, max_itr)

check_eig(T, ret[1])
display_all_real(ret[1])


expecting 255 pairs
Found 16 eigenpairs
Found 20 eigenpairs
Found 25 eigenpairs
Found 35 eigenpairs
Found 55 eigenpairs
Found 65 eigenpairs
Found 75 eigenpairs
Found 85 eigenpairs
Found 95 eigenpairs
Found 105 eigenpairs
Found 110 eigenpairs
Found 115 eigenpairs
Found 125 eigenpairs
Found 130 eigenpairs
Found 135 eigenpairs
Found 140 eigenpairs
Found 145 eigenpairs
Found 160 eigenpairs
Found 175 eigenpairs
Found 185 eigenpairs
Found 195 eigenpairs
Found 205 eigenpairs
Found 210 eigenpairs
Found 215 eigenpairs
Found 225 eigenpairs
Found 235 eigenpairs


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:73: RuntimeWarning: divide by zero encountered in double_scalars
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:83: RuntimeWarning: invalid value encountered in true_divide


Found 245 eigenpairs
Found 250 eigenpairs
Found 255 eigenpairs
tot time 112.374642 avg=0.440685
check lbd 0.000000
check equation 0.000000
check uniqueness
[0.00250032 0.00971076 0.02285533 0.02612621 0.02795656 0.0327449
 0.03713495 0.03821862 0.04054063 0.05312444 0.06042908 0.06132343
 0.0754039  0.08300173 0.09599904]


,lbd,0,1,2,3,4,5,6,7
6,0.245118,-0.006599,-0.267517,-0.095298,0.179163,-0.017914,-0.602854,0.120986,0.713315
13,0.633309,-0.233637,-0.456184,-0.158421,0.634026,0.097317,-0.415813,0.169090,0.315058
22,0.947505,-0.211168,0.299472,0.698460,-0.069450,0.431454,-0.366871,0.030050,-0.226726
41,0.942946,-0.223762,0.357706,0.613433,-0.148033,0.457172,-0.455372,0.000741,-0.085983
42,0.950609,-0.386343,0.382425,0.535561,-0.048909,0.418312,-0.486738,-0.053171,-0.023386
43,0.652108,-0.168748,0.057854,0.505665,-0.126866,0.054132,-0.629098,0.511098,0.190972
46,0.199743,-0.386549,0.777492,0.225334,-0.015238,-0.068042,0.313337,-0.285513,0.103686
51,0.182813,-0.572902,0.642630,0.294725,0.106883,-0.127189,0.293054,-0.139914,0.197204
52,0.659164,-0.132697,-0.025948,0.072521,-0.345635,0.006218,-0.556298,0.655866,0.342533
61,0.250817,-0.368665,0.565292,0.304970,0.158155,-0.158403,0.563265,-0.265551,-0.116770


# Summarizing the result of a matlab run in a table in the paper.

In [29]:
from IPython.display import display, HTML
def check_results_complex_pairs():
    from scipy.io import loadmat
    import pandas as pd

    res = loadmat('rayleigh_newton/matlab/matlab_save_unitary_res.mat')
    n_scenarios = res['save_res'].shape[0]
    # save_mean = np.zeros((n_scenarios, 3))
    # aggregate to table:
    # (m, n, j, # eigen values, #run time 90%, run time 100%
    # then run some pandas group by statements
    sum_table = pd.DataFrame(
        {'m': np.zeros((n_scenarios), dtype=int),
         'n': np.zeros((n_scenarios), dtype=int),
         'n_trys': np.zeros((n_scenarios), dtype=int),
         'n_pairs': np.zeros((n_scenarios), dtype=int),
         'n_real_pairs': np.zeros((n_scenarios), dtype=int),
         'n_self_conj_pairs': np.zeros((n_scenarios), dtype=int),
         'n_multiple_eigen': np.zeros((n_scenarios), dtype=int),
         'time_90': np.zeros((n_scenarios), dtype=float),
         'time_all': np.zeros((n_scenarios), dtype=float)},
        columns=['m', 'n', 'n_trys', 'n_pairs', 'n_real_pairs',
                 'n_self_conj_pairs', 'n_multiple_eigen',
                 'time_90', 'time_all'])
                             
    for j in range(n_scenarios):
        m = res['save_res'][j, 0][0, 0]
        n = res['save_res'][j, 0][0, 1]
        n_trys = res['save_res'][j, 0][0, 2]

        sum_table.loc[j, 'm'] = m
        sum_table.loc[j, 'n'] = n
        sum_table.loc[j, 'n_trys'] = n_trys
        eig_cell = res['save_res'][j, 1][0]

        dtypes = eig_cell.dtype
        dtypes_dict = dict((dtypes.names[a], a)
                           for a in range(len(dtypes.names)))

        n_pairs = res['save_res'][j, 1][0][0][dtypes_dict['lbd']].shape[0]
        sum_table.loc[j, 'n_pairs'] = n_pairs

        n_real_pairs = np.sum(
            res['save_res'][j, 1][0][0][dtypes_dict['is_real']])
        sum_table.loc[j, 'n_real_pairs'] = n_real_pairs

        n_self_conj_pairs = np.sum(
            res['save_res'][j, 1][0][0][dtypes_dict['is_self_conj']])
        sum_table.loc[j, 'n_self_conj_pairs'] = n_self_conj_pairs

        # find multiple eigen:
        # typically one lbd has one or two eigen vectors.
        # some cases we have multiple eigen vectors
        # we print them out here

        u, cnt = np.unique(['%.6f' % np.abs(a) for a in
                            res['save_res'][j, 1][0][0][dtypes_dict['lbd']]],
                           return_counts=True)
        dup_cnt = [(u[aa], cnt[aa]) for aa in range(len(u)) if cnt[aa] > 2]
        if len(dup_cnt) > 0:
            print('m=%d n=%d j=%d dup=%s' % (m, n, j, str(dup_cnt)))
            sum_table.loc[j, 'n_multiple_eigen'] = np.sum(
                [a[1] for a in dup_cnt])
        sum_table.loc[j, 'time_90'] = res['save_res'][j, 2][0, 0]
        sum_table.loc[j, 'time_all'] = res['save_res'][j, 3][0, 0]

    # sum_by_m_n = sum_table.groupby(['m', 'n']).sum()
    mean_by_m_n = sum_table.groupby(['m', 'n']).mean()
    mean_by_m_n.n_trys = sum_table[['m', 'n', 'n_trys']].groupby(
        ['m', 'n']).count()
    mean_by_m_n.n_pairs = sum_table[['m', 'n', 'n_pairs']].groupby(
        ['m', 'n']).mean()

    mean_by_m_n.n_real_pairs = sum_table[['m', 'n', 'n_real_pairs']].groupby(
        ['m', 'n']).mean()
    mean_by_m_n.n_multiple_eigen = sum_table[['m', 'n', 'n_multiple_eigen']].groupby(
        ['m', 'n']).mean()
    # mean_by_m_n.n_multiple_eigen = mean_by_m
    mean_by_m_n.time_90 /= mean_by_m_n.n_pairs
    mean_by_m_n.time_all /= mean_by_m_n.n_pairs
    mean_by_m_n.drop(columns=['n_self_conj_pairs'], inplace=True)

    # from IPython.core.display import display, HTML
    with open('sum.html', 'w') as hf:
        hf.write("%s\n" % mean_by_m_n.to_html())
    with open('mean.tex', 'w') as hf:
        hf.write("%s\n" % mean_by_m_n.to_latex())
        
    with open('sum_detail.html', 'w') as hf:
        hf.write("%s\n" % sum_table.to_html())     
    display(HTML(mean_by_m_n.to_html()))
check_results_complex_pairs()        

m=3 n=9 j=142 dup=[('0.734007', 3)]
m=4 n=8 j=280 dup=[('0.212739', 4), ('0.620611', 4)]
m=4 n=8 j=283 dup=[('0.533402', 4), ('0.583325', 4)]
m=4 n=8 j=284 dup=[('0.349782', 4)]
m=4 n=8 j=288 dup=[('0.405546', 4), ('0.411682', 4), ('0.719047', 4)]
m=4 n=8 j=289 dup=[('0.390941', 4)]
m=4 n=8 j=291 dup=[('0.075326', 4), ('0.476638', 3), ('0.578372', 4)]
m=4 n=8 j=292 dup=[('0.312587', 4), ('1.325293', 4)]
m=4 n=8 j=293 dup=[('1.080718', 4), ('1.259201', 4)]
m=4 n=8 j=296 dup=[('0.345709', 3), ('0.616980', 3)]
m=4 n=8 j=297 dup=[('0.392469', 4), ('0.533838', 4)]
m=4 n=8 j=298 dup=[('0.780948', 4)]
m=4 n=8 j=299 dup=[('0.888106', 4), ('0.897943', 4)]


# Comparing Schur and Newton implementation of RQI and Rayleigh Chebyshev algorithms for real tensor pair of 
$T(I, X^{[m-1]}) - \lambda X = 0$.

Also compare with $O-NCM$, which is Newton-RQI, but we do a bit of code optimization.

We found it is much faster to compute the orthognal complement using the Householder transform directly instead of using the nullspace function.

In [30]:
def householder(x):
  """ compute the orthogonal complement of x
  """
  n = x.shape[0]
  if x[0] < 0:
    al = 1
  else:
    al = -1
  I = np.eye(n)
  v = x - al*I[:, 0]
  v = v/norm(v)
  return I[:, 1:] - 2*v[:, None]@v[None, :]@I[:, 1:]

def newton_form_rayleigh_chebyshev2(
        T, max_itr, delta, x_init=None, do_chebyshev=True):
    """Newton form rayleigh
    """
    # get tensor dimensionality and order
    n_vec = T.shape
    m = len(n_vec)
    n = T.shape[0]
    R = 1
    converge = False

    # if not given as input, randomly initialize
    if x_init is None:
        x_init = np.random.randn(n)
        x_init = x_init/norm(x_init)

    # init lambda_(k) and x_(k)

    x_k = x_init.copy()
    if do_chebyshev:
        T_x_m_3 = symmetric_tv_mode_product(T, x_k, m-3)
        T_x_m_2 = np.tensordot(T_x_m_3, x_k, axes=1)
    else:
        T_x_m_2 = symmetric_tv_mode_product(T, x_k, m-2)
    T_x_m_1 = T_x_m_2 @ x_k
    lbd = x_k.T @ T_x_m_1
    ctr = 0

    while (R > delta) and (ctr < max_itr):
        # compute T(I,I,x_k,...,x_k), T(I,x_k,...,x_k) and g(x_k)

        #Q = get_xperp(x_k)
        # Q = null_space(x_k.reshape(1, -1))
        Q = householder(x_k)

        # compute Hessian H(x_k)
        H = (m-1)*T_x_m_2-lbd*np.eye(n)
        QTH = Q.T@H@Q
        y = -Q@solve(QTH, Q.T@(T_x_m_1-lbd * x_k))

        if do_chebyshev and (norm(y) < 5e-2):
            Rp_eta = y.T @ T_x_m_1 + (m-1) * x_k.T @ T_x_m_2 @ y -\
                2*(x_k.T @ y) * lbd
            L_x_lbd = - y * Rp_eta
            L_x_x = (m-1) * (m-2) * np.tensordot(T_x_m_3, y, axes=1) @ y
            T_a = Q@solve(QTH, Q.T@(2*L_x_lbd + L_x_x))
            x_k_n = x_k + y - 0.5*T_a
            x_k_n /= norm(x_k_n)
        else:
            x_k_n = (x_k + y) / norm(x_k + y)

        #  update residual and lbd
        R = norm(x_k-x_k_n)
        x_k = x_k_n
        if do_chebyshev:
            T_x_m_3 = symmetric_tv_mode_product(T, x_k, m-3)
            T_x_m_2 = np.tensordot(T_x_m_3, x_k, axes=1)
        else:
            T_x_m_2 = symmetric_tv_mode_product(T, x_k, m-2)
        T_x_m_1 = T_x_m_2 @ x_k

        lbd = np.sum(x_k * T_x_m_1)
        ctr += 1
    x = x_k
    err = norm(symmetric_tv_mode_product(
        T, x, m-1) - lbd * x)
    if ctr < max_itr:
        converge = True

    return x, lbd, ctr, converge, err

In [31]:
def test_eigen_tensor(k, m, max_err, max_itr, n_test):
    A = utils.generate_symmetric_tensor(k, m)

    o_ncm_cnt = np.full(n_test, fill_value=np.nan)
    schur_cnt = np.full(n_test, fill_value=np.nan)
    newton_cnt = np.full(n_test, fill_value=np.nan)
    schur_cheb_cnt = np.full(n_test, fill_value=np.nan)

    o_ncm_err = np.full(n_test, fill_value=np.nan)
    schur_err = np.full(n_test, fill_value=np.nan)
    newton_err = np.full(n_test, fill_value=np.nan)
    schur_cheb_err = np.full(n_test, fill_value=np.nan)

    o_ncm_lbd = np.full(n_test, fill_value=np.nan)
    schur_lbd = np.full(n_test, fill_value=np.nan)
    newton_lbd = np.full(n_test, fill_value=np.nan)
    schur_cheb_lbd = np.full(n_test, fill_value=np.nan)

    o_ncm_time = np.full(n_test, fill_value=np.nan)
    schur_time = np.full(n_test, fill_value=np.nan)
    newton_time = np.full(n_test, fill_value=np.nan)
    schur_cheb_time = np.full(n_test, fill_value=np.nan)

    for jj in range(n_test):
        x0 = np.random.randn(k)
        x0 = x0 / np.linalg.norm(x0)

        # do orthogonal
        t_start = process_time()
        o_x, o_lbd, o_ctr, converge = orthogonal_newton_correction_method(
            A, max_itr, max_err, x_init=x0)
        t_end = process_time()
        if converge:
          o_ncm_cnt[jj] = o_ctr
          o_ncm_lbd[jj] = o_lbd
          o_ncm_err[jj] = np.linalg.norm(
              symmetric_tv_mode_product(
                  A, o_x, m-1) - o_lbd * o_x)
          o_ncm_time[jj] = t_end - t_start

        # do schur_form_rayleigh
        t_start = process_time()
        # s_x, s_lbd, ctr, converge = schur_form_rayleigh_chebyshev_linear(
        # A, max_itr, max_err, x_init=x0, do_chebyshev=True)
        s_x, s_lbd, ctr, converge, err = schur_form_rayleigh_chebyshev(
            A, max_itr, max_err, x_init=x0, do_chebyshev=False)

        t_end = process_time()
        if converge:
          schur_cnt[jj] = ctr
          schur_lbd[jj] = s_lbd
          schur_err[jj] = np.linalg.norm(
              symmetric_tv_mode_product(
                  A, s_x, m-1) - s_lbd * s_x)
          schur_time[jj] = t_end - t_start

        # now do rayleigh
        t_start = process_time()
        ntn_x, ntn_lbd, ctr, converge, err = newton_form_rayleigh_chebyshev2(
              A, max_itr, max_err, x_init=x0, do_chebyshev=False)
        t_end = process_time()
        if converge:
          newton_time[jj] = t_end - t_start
          newton_cnt[jj] = ctr
          newton_lbd[jj] = ntn_lbd
          newton_err[jj] = np.linalg.norm(
              symmetric_tv_mode_product(
                  A, ntn_x, m-1) - ntn_lbd * ntn_x)

        # now do rayleigh chebyshev
        t_start = process_time()        
        sch_x, sch_lbd, ctr, converge, err = schur_form_rayleigh_chebyshev(
              A, max_itr, max_err, x_init=x0, do_chebyshev=True)

        t_end = process_time()
        """
        res_newton_cheb = rayleigh_chebyshev(
            e, x0, max_err=max_err, max_iter=max_itr,
            verbose=False, exit_by_diff=True)
        """
        if converge:
          schur_cheb_time[jj] = t_end - t_start
          schur_cheb_cnt[jj] = ctr
          schur_cheb_lbd[jj] = sch_lbd
          schur_cheb_err[jj] = np.linalg.norm(
              symmetric_tv_mode_product(
                  A, sch_x, m-1) - sch_lbd * sch_x)
          schur_cheb_time[jj] = t_end - t_start

    summ = pd.DataFrame(
        {
            'o_ncm_iter': o_ncm_cnt,
            'schur_iter': schur_cnt,
            'newton_iter': newton_cnt, 'schur_cheb_iter': schur_cheb_cnt,
            'o_ncm_err': o_ncm_err,
            'schur_err': schur_err,
            'newton_err': newton_err, 'schur_cheb_err': schur_cheb_err,
            'o_ncm_lbd': o_ncm_lbd,
            'schur_lbd': schur_lbd,
            'newton_lbd': newton_lbd,
            'schur_cheb_lbd': schur_cheb_lbd,
            'o_ncm_time': o_ncm_time,
            'schur_time': schur_time,
            'newton_time': newton_time,
            'schur_cheb_time': schur_cheb_time
        },
        columns=['o_ncm_iter', 'o_ncm_lbd', 'o_ncm_err', 'o_ncm_time',
                 'schur_iter', 'schur_lbd', 'schur_err', 'schur_time',
                 'newton_iter', 'newton_lbd', 'newton_err', 'newton_time',
                 'schur_cheb_iter', 'schur_cheb_lbd', 'schur_cheb_err',
                 'schur_cheb_time'])
    return summ

In [32]:

np.random.seed(0)
k = 6
m = 3
max_err = 1e-10
max_itr = 200
n_test = 1000

summ = test_eigen_tensor(k, m, max_err, max_itr, n_test)
# summ[['o_ncm_time', 'schur_time', 'newton_time', 'newton_cheb_time']].describe())
# display(HTML(summ.describe().to_html()))
display(HTML(summ[[a for a in summ.columns if 'time' in a]].describe().to_html()))
display(HTML(summ[[a for a in summ.columns if 'iter' in a]].describe().to_html()))
display(HTML(summ[[a for a in summ.columns if 'lbd' in a]].describe().to_html()))
display(HTML(summ[[a for a in summ.columns if 'err' in a]].describe().to_html()))


,o_ncm_time,schur_time,newton_time,schur_cheb_time
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.008724,0.003987,0.004562,0.004882
std,0.006201,0.002862,0.003149,0.002547
min,0.003012,0.001441,0.001652,0.001902
25%,0.004875,0.002237,0.002606,0.003160
50%,0.006411,0.002927,0.003413,0.004081
75%,0.009959,0.004560,0.005278,0.005759
max,0.051579,0.021528,0.022212,0.020570


,o_ncm_iter,schur_iter,newton_iter,schur_cheb_iter
count,1000.0000,1000.000000,1000.000000,1000.000000
mean,13.7230,13.783000,13.702000,11.483000
std,10.1016,10.374369,9.989744,7.211915
min,5.0000,5.000000,5.000000,4.000000
25%,8.0000,8.000000,8.000000,7.000000
50%,10.0000,10.000000,10.000000,9.000000
75%,16.0000,16.000000,16.000000,14.000000
max,87.0000,83.000000,75.000000,55.000000


,o_ncm_lbd,schur_lbd,newton_lbd,schur_cheb_lbd
count,1000.000000,1000.000000,1000.000000,1000.000000
mean,0.000939,-0.019187,-0.013954,0.001127
std,1.657999,1.579579,1.618327,1.472290
min,-8.041008,-8.041008,-8.041008,-8.041008
25%,-0.710710,-0.710710,-0.710710,-0.710710
50%,-0.000731,-0.000731,-0.000731,-0.000731
75%,0.710710,0.710710,0.710710,0.710710
max,8.041008,8.041008,8.041008,8.041008


,o_ncm_err,schur_err,newton_err,schur_cheb_err
count,1.000000e+03,1.000000e+03,1.000000e+03,1.000000e+03
mean,2.636017e-16,3.667934e-16,2.630650e-16,3.500143e-16
std,3.065191e-16,4.555482e-16,3.440989e-16,4.232397e-16
min,2.715272e-17,4.330081e-17,3.141718e-17,4.166949e-17
25%,1.405420e-16,1.888858e-16,1.401373e-16,1.862870e-16
50%,2.000851e-16,2.611270e-16,1.934700e-16,2.695539e-16
75%,2.857822e-16,3.863391e-16,2.733607e-16,3.816559e-16
max,3.044522e-15,4.094300e-15,3.293454e-15,4.925188e-15
